In [1]:
import fiona
from madmex.models import PredictObject
from madmex.models import PredictClassification, Region, ValidClassification
import json
from shapely.geometry import mapping
from madmex.util.spatial import geometry_transform
from shapely.geometry import shape
import os
from operator import itemgetter
from django.contrib.gis.geos.geometry import GEOSGeometry
from fiona.crs import to_string
from dask.distributed import Client

In [2]:
predict_name = 'lc_rf_s1_2_20m_resampled_10m_Jalisco_recipe_2017'

In [3]:
qs_ids = PredictClassification.objects.filter(name=predict_name).distinct('predict_object_id')

In [4]:
list_ids = [x.predict_object_id for x in qs_ids]

In [5]:
len(list_ids)

56

In [6]:
region = 'Jalisco'

In [7]:
region_geom = Region.objects.get(name=region).the_geom
region_geojson = region_geom.geojson
geometry_region = json.loads(region_geojson)
#proj4='+proj=lcc +lat_1=17.5 +lat_2=29.5 +lat_0=12 +lon_0=-102 +x_0=2500000 +y_0=0 +a=6378137 +b=6378136.027241431 +units=m +no_defs'
#geometry_region_proj = geometry_transform(geometry_region,proj4)

In [8]:
validation_set='validation_Jalisco'

In [10]:
def valid_object_to_feature(x, crs=None):
    attr = {'class': x.valid_tag.numeric_code}
    if crs is None:
        geometry = json.loads(x.valid_object.the_geom.geojson)
    else:
        geometry = json.loads(x.valid_object.the_geom.transform(crs, clone=True).geojson)
    feature = {
        "type": "Feature",
        "geometry": geometry,
        "properties": attr
    }
    return feature

In [11]:
def fun(id_dc_tile, predict_name, geometry_region_proj):
    seg = PredictObject.objects.filter(id=id_dc_tile)
    s3_path = seg[0].path
    poly = seg[0].the_geom
    #next lines to reproyect extent registered in DB TODO: register geometry of 
    #extent of each dc tile in lat long
    poly_geojson = poly.geojson
    geometry = json.loads(poly_geojson)
    with fiona.open(s3_path) as src:
        crs = to_string(src.crs)
        poly = GEOSGeometry(json.dumps(geometry))
        qs_dc_tile = ValidClassification.objects.filter(valid_object__the_geom__contained=poly,
                                                   valid_set=validation_set).prefetch_related('valid_object', 'valid_tag') 
    
        fc_qs = [valid_object_to_feature(x) for x in qs_dc_tile]
        if geometry_region is not None:
            shape_region=shape(geometry_region)
            fc_qs_in_region = [{'geometry': mapping(shape_region.intersection(shape(x['geometry']))),
                                'class': x['properties']['class']} for x in fc_qs if shape_region.intersects(shape(x['geometry']))] 
            fc_qs = fc_qs_in_region
            fc_qs_in_region = None 
        fc_qs_proj = [feature_transform(x, crs_out=crs) for x in fc_qs]
        fc_qs_proj = [(x['geometry'],x['class']) for x in fc_qs_proj]
        #create fc with (geometry, tag) values
        pred_objects_sorted = PredictClassification.objects.filter(name=predict_name,
                                                                   predict_object_id=id_dc_tile).prefetch_related('tag').order_by('features_id')
        fc_pred=[(x['properties']['id'], x['geometry']) for x in src]
        fc_pred_sorted = sorted(fc_pred, key=itemgetter(0))
        fc_pred = [(x[0][1], x[1].tag.numeric_code) for x in zip(fc_pred_sorted, pred_objects_sorted)]
        fc_pred_sorted = None
        pred_objects_sorted = None
        #intersect with fc of validation set
        fc_pred_intersect_validset = [(x[0],x[1]) for x in fc_pred for y in fc_qs_proj if shape(x[0]).intersects(shape(y[0]))]
        fc_pred = None
    return [fc_qs_proj, fc_pred_intersect_validset]

In [12]:
scheduler_file= '/shared_volume/scheduler.json'

In [13]:
client = Client(scheduler_file=scheduler_file)

In [14]:
client

Client Scheduler: tcp://100.96.3.14:8786 Dashboard: http://100.96.3.14:8787/status,Cluster Workers: 3 Cores: 3 Memory: 36.00 GB


In [15]:
client.restart()

Client Scheduler: tcp://100.96.3.14:8786 Dashboard: http://100.96.3.14:8787/status,Cluster Workers: 3 Cores: 3 Memory: 36.00 GB


In [16]:
c = client.map(fun,list_ids,**{'predict_name': predict_name,
                              'geometry_region_proj': geometry_region_proj})

In [17]:
result = client.gather(c)

In [18]:
fc_valid_result = [x[0][index] for x in result for index in range(0,len(x[0]))]

In [19]:
fc_test_result = [x[1][index] for x in result for index in range(0,len(x[1]))]

In [20]:
len(fc_valid_result)

245

In [21]:
len(fc_test_result)

1564

In [22]:
from madmex.validation import prepare_validation

In [23]:
y_true, y_pred, sample_weight = prepare_validation(fc_valid_result, fc_test_result)

In [24]:
len(y_true)

1587

In [25]:
len(y_pred)

1587

In [26]:
from madmex.validation import validate

In [27]:
acc_dict = validate(y_true=y_true, y_pred=y_pred, sample_weight=sample_weight,
                    scheme='madmex_31')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
from madmex.validation import pprint_val_dict

In [29]:
pprint_val_dict(acc_dict)

Numeric code    User's Accuracy      Producer's Accuracy  Class Name                                        
1               0.00                 0.00                                                                   
2               0.17                 0.49                                                                   
3               0.39                 0.50                                                                   
5               0.00                 0.00                                                                   
6               0.71                 0.02                                                                   
7               0.00                 0.00                                                                   
8               0.77                 0.83                                                                   
11              0.51                 0.52                                                                   
12              0.5